In [1]:
# Imports
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import base64
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import pickle
from surprise import Dataset, Reader, SVD

# Paths (Windows)
image_filename1 = r"C:\Users\hp\OneDrive\Desktop\skincare_project\output 1.png"
image_filename2 = r"C:\Users\hp\OneDrive\Desktop\skincare_project\output 2.png"
csv_path = r"C:\Users\hp\OneDrive\Desktop\skincare_project\skindataall.csv"
mf_model_path = r"C:\Users\hp\OneDrive\Desktop\skincare_project\mf_model.pkl"

# Encode images
encoded_image1 = base64.b64encode(open(image_filename1, 'rb').read())
encoded_image2 = base64.b64encode(open(image_filename2, 'rb').read())

# Load CSV and model
df = pd.read_csv(csv_path, encoding='ISO-8859-1', index_col=[0])
with open(mf_model_path, 'rb') as f:
    mf_model = pickle.load(f)

# Skin concern keywords
concern_keywords = {
    'Acne': ['salicylic', 'benzoyl', 'tea tree'],
    'Dryness': ['hyaluronic', 'glycerin', 'ceramide'],
    'Aging': ['retinol', 'peptide', 'vitamin c'],
    'Sensitivity': ['aloe', 'chamomile', 'niacinamide']
}

# Helper functions
def dicts(df, colname):
    vals = list(set(df[colname]))
    return [{'label': i, 'value': i} for i in vals]

def Table(df):
    """Return a Dash HTML table from a DataFrame."""
    rows = []
    for i in range(len(df)):
        row = []
        for col in df.columns:
            if col == 'Product_Url':
                continue
            value = df.iloc[i][col]
            if col == 'Product':
                cell = html.Td(html.A(href=df.iloc[i]['Product_Url'], children=value))
            else:
                cell = html.Td(value)
            row.append(cell)
        rows.append(html.Tr(row))
    headers = [html.Th(col) for col in df.columns if col != 'Product_Url']
    return html.Table([html.Tr(headers)] + rows)

def create_interaction_matrix(df, user_col, item_col, rating_col, norm=False, threshold=None):
    interactions = df.groupby([user_col, item_col])[rating_col].sum().unstack().reset_index().fillna(0).set_index(user_col)
    if norm:
        interactions = interactions.applymap(lambda x: 1 if x > threshold else 0)
    return interactions

def create_user_dict(interactions):
    return {user_id: i for i, user_id in enumerate(interactions.index)}

def create_item_dict(df, id_col, name_col):
    return {df.loc[i, id_col]: df.loc[i, name_col] for i in df.index}

# Dictionaries
tones_dict = dicts(df, 'Skin_Tone')
types_dict = dicts(df, 'Skin_Type')
eyes_dict = dicts(df, 'Eye_Color')
hair_dict = dicts(df, 'Hair_Color')
products_dictionary = dicts(df, 'Product')
user_dictionary = dicts(df, 'User_id')

interaction_matrix = create_interaction_matrix(df, 'User_id', 'Product_id', 'Rating_Stars')
user_dict = create_user_dict(interaction_matrix)
product_dict = create_item_dict(df, 'Product_id', 'Product')

# Dash app setup
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

colors = {
    "text": "#111111",
    "background-image": f"url({image_filename1})",
    "background-size": "cover",
}

# Layout
app.layout = html.Div(style=colors, children=[
    html.H1('Skincare Recommendations', style={'textAlign': 'center', 'color': colors['text'], 'font-family': 'Bangers'}),
    dcc.Markdown('''__Welcome to our revolutionary skincare recommendation model...__'''),
    dcc.Markdown('---'),

    html.Label('Skin Tone'), dcc.Dropdown(id='skintone-selector', options=tones_dict, placeholder='Select your skin tone'),
    html.Label('Skin Type'), dcc.Dropdown(id='skintype-selector', options=types_dict, placeholder='Select your skin type'),
    html.Label('Eye color'), dcc.Dropdown(id='eyecolor-selector', options=eyes_dict, placeholder='Select your eye color'),
    html.Label('Hair color'), dcc.Dropdown(id='haircolor-selector', options=hair_dict, placeholder='Select your hair color'),

    html.Label('Skin Concern'),
    dcc.Dropdown(id='concern-selector', options=[{'label': k, 'value': k} for k in concern_keywords.keys()], placeholder='Select your skin concern'),

    dcc.Markdown('''__Based on your features and skin concern, these are the top products for you:__'''),
    html.Div(id='output_1'),

    dcc.Markdown('---'),
    html.H2('Skincare recommendations based on your favorites', style={'textAlign': 'center', 'color': colors['text'], 'font-family': 'Bangers'}),
    html.Label('Your favorite product!'), dcc.Dropdown(id='product-selector', options=products_dictionary, placeholder='Select your favorite product'),
    dcc.Markdown('''__Based on your preference, these are the top products for you:__'''),
    html.Div(id='output_2'),

    dcc.Markdown('---'),
    html.H2('Skincare recommendations to users (for business)', style={'textAlign': 'center', 'color': colors['text'], 'font-family': 'Bangers'}),
    html.Label('List of user ids'), dcc.Dropdown(id='user-selector', options=user_dictionary, placeholder='Select user'),
    dcc.Markdown('''__This user may like the following products:__'''),
    html.Div(id='output_3')
])

# Callbacks
@app.callback(
    Output('output_1', 'children'),
    [Input('skintone-selector', 'value'),
     Input('skintype-selector', 'value'),
     Input('eyecolor-selector', 'value'),
     Input('haircolor-selector', 'value'),
     Input('concern-selector', 'value')]
)
def recommend_by_features_and_concern(skintone, skintype, eyecolor, haircolor, concern):
    try:
        ddf = df[(df['Skin_Tone'] == skintone) &
                 (df['Hair_Color'] == haircolor) &
                 (df['Skin_Type'] == skintype) &
                 (df['Eye_Color'] == eyecolor)]

        if concern:
            keywords = concern_keywords.get(concern, [])
            ddf = ddf[ddf['Ingredients'].apply(lambda x: any(k in str(x).lower() for k in keywords))]

        recommendations = ddf[ddf['Rating_Stars'].notnull()]
        if recommendations.empty:
            return html.Div("No recommendations found for your selection. Please try different options.", style={'color':'red'})

        data = recommendations[['Rating_Stars', 'Product_Url', 'Product']]
        data = data.sort_values('Rating_Stars', ascending=False).head()
        return Table(data)

    except Exception as e:
        return html.Div(f"Error: {str(e)}", style={'color': 'red'})


@app.callback(
    Output('output_2', 'children'),
    [Input('product-selector', 'value')]
)
def content_recommender(product):
    try:
        if not product:
            return html.Div("Please select a product.", style={'color':'red'})

        df_cont = df[['Product', 'Product_id', 'Ingredients', 'Product_Url']].drop_duplicates().reset_index(drop=True)
        tf = TfidfVectorizer(analyzer='word', ngram_range=(1,2), min_df=1, stop_words='english')
        tfidf_matrix = tf.fit_transform(df_cont['Ingredients'])
        cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

        indices = pd.Series(df_cont.index, index=df_cont['Product'])
        idx = indices[product]

        sim_scores = list(enumerate(cosine_sim[idx]))
        sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:11]
        product_indices = [i[0] for i in sim_scores]

        return Table(df_cont.iloc[product_indices][['Product', 'Product_Url']])

    except Exception as e:
        return html.Div(f"Error: {str(e)}", style={'color':'red'})


@app.callback(
    Output('output_3', 'children'),
    [Input('user-selector', 'value')]
)
def sample_recommendation_user(user_id, model=mf_model, interactions=interaction_matrix, user_dict=user_dict, item_dict=product_dict, threshold=4, nrec_items=10):
    try:
        if not user_id:
            return html.Div("Please select a user.", style={'color':'red'})

        n_users, n_items = interactions.shape
        user_x = user_dict[user_id]
        scores = pd.Series(model.predict(user_x, np.arange(n_items)))
        scores.index = interactions.columns
        scores = list(pd.Series(scores.sort_values(ascending=False).index))

        known_items = list(pd.Series(interactions.loc[user_id, :][interactions.loc[user_id, :] > threshold].index).sort_values(ascending=False))
        scores = [x for x in scores if x not in known_items]
        return_score_list = scores[:nrec_items]

        known_items = [item_dict[i] for i in known_items]
        scores = [item_dict[i] for i in return_score_list]

        return [html.H6(i) for i in scores]

    except Exception as e:
        return html.Div(f"Error: {str(e)}", style={'color':'red'})


if __name__ == '__main__':
    app.run(debug=True, port=8051)



C:\Users\hp\skincare_env\Lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(
